In [ ]:
from langchain.schema.runnable import RunnableSequence,RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os
import json
os.environ["OPENAI_API_KEY"] = ""
llm = ChatOpenAI(
    model="gpt-4o-mini"
)

In [2]:
def validate_order(order):
    """
        驗證訂單系統
    """
    errors=[]
    if not order.get("customer_id"):
        errors.append("缺少客戶id")
    if not order.get("items") or len(order["items"]) == 0:
        errors.append("訂單中沒有商品")
    return {"order":order,"is_valid":len(errors)==0,"errors":errors}

order_RunnableLambda=RunnableLambda(validate_order)

In [3]:
def prepare_llm_input(processed_order):
    """
    準備LLM輸入
    """
    return {"order_info":json.dumps(processed_order,ensure_ascii=False)}

prepare_llm_input_RunnableLambda=RunnableLambda(prepare_llm_input)

In [4]:
summary_prompt=ChatPromptTemplate.from_template(
    "你是一個電子商品平台的客戶服務助手。請根據以下訂單內容生成訂單摘要。"
    "如果訂單無效，請解釋原因。訂單內容:### {order_info}###"
)

summary_chain=summary_prompt | llm

In [5]:
workflow=RunnableSequence(
    validate_order,
    prepare_llm_input_RunnableLambda,
    summary_chain,
    StrOutputParser()
)

In [6]:
test_orders=[
    {
        "customer_id":"CUS001",
        "items":[
            {"name":"筆記本電腦","price":3500,"quantity":1},
            {"name":"滑鼠","price":2500,"quantity":2}
        ]
    },
    {
        "customer_id":"CUS002",
        "items":[]
    },
    {
        "items":[
            {"name":"鍵盤","price":500,"quantity":1}
        ]
    }
]

for order in test_orders:
    result=workflow.invoke(order)
    print(result)
    print("-------------------------------------------------------------")

訂單摘要：

- **客戶ID**: CUS001
- **訂單有效性**: 有效
- **商品明細**:
  - **筆記本電腦**: 價格 3500 元, 數量 1
  - **滑鼠**: 價格 2500 元, 數量 2

若您有任何其他問題或需要進一步的幫助，請隨時告知！
-------------------------------------------------------------
訂單摘要：

- 客戶ID：CUS002
- 訂單狀態：無效
- 錯誤原因：訂單中沒有商品

由於該訂單未包含任何商品，因此無法處理。請添加商品後再次提交訂單。
-------------------------------------------------------------
訂單摘要：

- 訂單項目：鍵盤
- 價格：500元
- 數量：1

**訂單狀態：無效**

**原因：** 此訂單缺少客戶ID，無法處理。請提供有效的客戶ID以便完成訂單。
-------------------------------------------------------------
